# BytePair Encoding

In [3]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.1 MB/s eta 0:00:00


In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = "hello, this is JJ from Tehran."

token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(token_ids)

[31373, 11, 428, 318, 38775, 422, 20882, 13]


In [5]:
strings = tokenizer.decode(token_ids)

print(strings)

hello, this is JJ from Tehran.


## data sampling with sliding window

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader


# A dataset for batched inputs and targets
class GPTDatasetV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []
    token_ids = tokenizer.encode(text)

    for i in range(0, len(token_ids) - max_length, stride):
      self.input_ids.append(torch.tensor(token_ids[i:i+max_length]))
      self.target_ids.append(torch.tensor(token_ids[i+1:i+max_length+1]))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [8]:
# A data loader to generate batches with input-with pairs

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, num_workers=0, shuffle=True, drop_last=True):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader

## encoding word position

In [9]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
raw_text, batch_size=8, max_length=max_length,
stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
token_embeddings = token_embedding_layer(inputs)


context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))


